In [1]:
import datetime
import itertools
import concurrent.futures
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import random
from scipy.optimize import minimize
from scipy import linalg
import seaborn as sns
import time

data = pd.read_csv("../data/backtest-2.csv", index_col="date")
# # organize DataSeries
# date = data.index
# tLen = len(data)
# time = np.linspace(0, tLen-1, tLen)
# close = [data["Adj Close"][i] for i in range(len(data["Adj Close"]))]
# DataSeries = [time, close]

# matrix helpers
def _yi(price_series):
    return [np.log(p) for p in price_series]

def _fi(tc, m, time_series):
    return [np.power((tc - t), m) for t in time_series]

def _gi(tc, m, w, time_series):
    return [np.power((tc - t), m) * np.cos(w * np.log(tc - t)) for t in time_series]

def _hi(tc, m, w, time_series):
    return [np.power((tc - t), m) * np.sin(w * np.log(tc - t)) for t in time_series]

def _fi_pow_2(tc, m, time_series):
    return np.power(_fi(tc, m, time_series), 2)

def _gi_pow_2(tc, m, w, time_series):
    return np.power(_gi(tc, m, w, time_series), 2)

def _hi_pow_2(tc, m, w, time_series):
    return np.power(_hi(tc, m, w, time_series), 2)

def _figi(tc, m, w, time_series):
    return np.multiply(_fi(tc, m, time_series), _gi(tc, m, w, time_series))

def _fihi(tc, m, w, time_series):
    return np.multiply(_fi(tc, m, time_series), _hi(tc, m, w, time_series))

def _gihi(tc, m, w, time_series):
    return np.multiply(_gi(tc, m, w, time_series), _hi(tc, m, w, time_series))

def _yifi(tc, m, time_series, price_series):
    return np.multiply(_yi(price_series), _fi(tc, m, time_series))

def _yigi(tc, m, w, time_series, price_series):
    return np.multiply(_yi(price_series), _gi(tc, m, w, time_series))

def _yihi(tc, m, w, time_series, price_series):
    return np.multiply(_yi(price_series), _hi(tc, m, w, time_series))

In [2]:
# revised version of the LPPL without φ
# found on page 11 as equation (13)
def lppl(t, tc, m, w, a, b, c1, c2):
    return a + np.power(tc - t, m) * (b + ((c1 * np.cos(w * np.log(tc - t))) + (c2 * np.sin(w * np.log(tc - t)))))

# finds the least square difference
def func_restricted(x, *args):
    tc = x[0]
    m  = x[1]
    w  = x[2]
    
    data_series = args[0]
    
    lin_vals = matrix_equation(tc, m, w, data_series)
    
    a  = lin_vals[0] 
    b  = lin_vals[1]
    c1 = lin_vals[2] 
    c2 = lin_vals[3]
    
    delta = [lppl(t, tc, m, w, a, b, c1, c2) for t in data_series[0]]
    delta = np.subtract(delta, data_series[1])
    delta = np.power(delta, 2)
    return np.sum(delta)


# solve the matrix equation
def matrix_equation(tc, m, w, data_series):
    time_series = data_series[0]
    price_series = data_series[1]
    N  = len(price_series)
    
    fi = np.sum(_fi(tc, m, time_series))
    gi = np.sum(_gi(tc, m, w, time_series))
    hi = np.sum(_hi(tc, m, w, time_series))
    
    fi_pow_2 = np.sum(_fi_pow_2(tc, m, time_series))
    gi_pow_2 = np.sum(_gi_pow_2(tc, m, w, time_series))
    hi_pow_2= np.sum(_hi_pow_2(tc, m, w, time_series))
    
    figi = np.sum(_figi(tc, m, w, time_series))
    fihi = np.sum(_fihi(tc, m, w, time_series))
    gihi = np.sum(_gihi(tc, m, w, time_series))
    
    yi = np.sum(_yi(price_series))
    yifi = np.sum(_yifi(tc, m, time_series, price_series))
    yigi = np.sum(_yigi(tc, m, w, time_series, price_series))
    yihi = np.sum(_yihi(tc, m, w, time_series, price_series))
    
    matrix_1 = np.matrix([
        [N,  fi,       gi,       hi      ],
        [fi, fi_pow_2, figi,     fihi    ],
        [gi, figi,     gi_pow_2, gihi    ],
        [hi, fihi,     gihi,     hi_pow_2]
    ])
    
    matrix_2 = np.matrix([
        [yi],
        [yifi],
        [yigi],
        [yihi]
    ])
    
    product = np.linalg.solve(matrix_1, matrix_2)
    
    return [i[0] for i in product.tolist()]

In [3]:
def compute_ds_lppls_confidence(n, s):
    
    tLen = 180-(n*5)
#     print(tLen)
    tradings_days_data = data.tail(tLen)
    time = np.linspace(0, tLen-1, tLen)
    price = [tradings_days_data[s][i] for i in range(len(tradings_days_data[s]))]
    data_series = [time, price]

    found_solution = False

    while not found_solution:

        # set limits for non-linear params
        limits = (
            [tLen-(tLen*0.2), tLen+(tLen*0.2)],    # Critical Time + or - .2
#             [-0.2, 0.2],
            [0.1, 0.9],                            # m : 0.1 ≤ m ≤ 0.9
            [6, 13],                               # ω : 6 ≤ ω ≤ 13     
        )

        # randomly choose vals for non-linear params 
        non_lin_vals = [random.uniform(a[0], a[1]) for a in limits]

        tc = non_lin_vals[0]
        m  = non_lin_vals[1] 
        w  = non_lin_vals[2]

        # params to pass to scipy.optimize
        seed = [tc, m, w]

        # scipy optimize minimize
        try:
            cofs = minimize(fun=func_restricted, x0=seed, args=(data_series), method='Nelder-Mead')

            if cofs.success:
                # determine if it falls in range:

                tc = cofs.x[0]
                m =  cofs.x[1]
                w =  cofs.x[2]

                tc_in_range = tLen-(tLen*0.05) < tc < tLen+(tLen*0.1)
                m_in_range  = 0.01 < m < 1.2
                w_in_range  = 2 < w < 25
                n_oscillation = ((w/2)*np.log(abs((tc - (n*5))/(tLen)))) > 2.5
#                 print("tc: {}".format(tc))
#                 print("tLen: {}".format(tLen))
#                 print("tStart: {}".format(n*5))
#                 print("tEnd: {}".format(180))
#                 print("___________")
#                 damping = ""
#                 relative_error = ""

                if (tc_in_range and m_in_range and w_in_range and n_oscillation):
                    ds_lppls_confidence = True
                else: 
                    ds_lppls_confidence = False

                return {
                    'ds_lppls_confidence': ds_lppls_confidence,
                    'cof': cofs.x,
                }
        except Exception as e:
            print(e)
    
def main(time_range, s):
    solutions = []
    for i in range(time_range):
        
        
        #     with concurrent.futures.ProcessPoolExecutor() as executor:
        #         for foo in executor.map(compute_ds_lppls_confidence, time_range, s):
        #             solutions.append(foo)

        foo = compute_ds_lppls_confidence(i, s)
        solutions.append(foo)
        
    true_count = 0
    total_count = len(solutions)
    for i in solutions:
        if i["ds_lppls_confidence"] == True:
            true_count = true_count + 1

#     print("{}/{}={}".format(true_count,total_count,true_count/total_count))
    return (true_count/total_count)


In [4]:
ds_lppls = []

for s in data.columns:
    print("################")
    print("#      {}      #".format(s))
    print("################")
#     TIME_RANGE = [i for i in range(31)]
#     data = pd.read_csv("../data/backtest.csv", index_col="date")
    # print(data.tail())
#     data.drop(data.tail(i).index,inplace=True)
    # print(data.tail())
    start = time.time()
    ds_lppl = main(31, s)
    end = time.time()
    ds_lppls.append({s: ds_lppl})
    print(end - start)
    print(ds_lppls)



################
#      A      #
################
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands cou

/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in power
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in power
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in log
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in power
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in log


operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (18

operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (18

operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (18

...

operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (18

operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (18

operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (18

operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (180,) 
operands could not be broadcast together with shapes (64,) (18

KeyboardInterrupt: 

In [ ]:
print(data.columns)